In [1]:
import pandas as pd

In [2]:
def primera_llamada(name):

    df = pd.read_csv('ibex35.csv')

    name = df[df['empresa'] == name]

    name = name [['date','open','high','low','close','adj close']]

    name['date'] = pd.to_datetime(name['date'], format="%Y-%m-%d")

    name.sort_values(by='date',ascending= True, inplace=True)

    fecha_inicio = '2021-01-01'
    fecha_fin = '2024-03-01'

    name = pd.DataFrame(name[(name['date'] >= fecha_inicio) & (name['date'] <= fecha_fin)]).reset_index(drop=True)

    return name

In [3]:
acerinox = primera_llamada('Acerinox')

acerinox

,date,open,high,low,close,adj close
0,2021-01-04,9.180,9.432,9.142,9.294,7.683368
1,2021-01-05,9.310,9.334,9.214,9.312,7.698248
2,2021-01-06,9.334,9.546,9.304,9.538,7.885085
3,2021-01-07,9.590,9.840,9.550,9.820,8.118213
4,2021-01-08,9.900,9.900,9.670,9.712,8.028930
...,...,...,...,...,...,...
807,2024-02-26,10.245,10.255,10.130,10.230,10.230000
808,2024-02-27,10.260,10.400,10.250,10.400,10.400000
809,2024-02-28,10.365,10.395,10.175,10.200,10.200000
810,2024-02-29,10.245,10.260,10.095,10.135,10.135000


In [4]:

def preparar_modelo(DF):


    DF_filt = DF[['date','close']]

    DF_filt.close = DF.close.round(2)

    DF_filt= DF_filt.reset_index(drop=True)

    new_df = pd.DataFrame(columns=['d-4', 'd-3', 'd-2', 'd-1', 'do'])

    for i in range(len(new_df.columns)):
        new_df[new_df.columns[i]] = DF_filt['close'].shift(-i)

    new_df['date']= DF_filt.date

    new_df.insert(0, 'date', new_df.pop('date'))

    new_df.dropna(axis=0,inplace=True)

    return new_df



In [5]:
new_df = preparar_modelo(acerinox)

new_df

C:\Users\javie\AppData\Local\Temp\ipykernel_37428\4040868792.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_filt.close = DF.close.round(2)


,date,d-4,d-3,d-2,d-1,do
0,2021-01-04,9.29,9.31,9.54,9.82,9.71
1,2021-01-05,9.31,9.54,9.82,9.71,9.70
2,2021-01-06,9.54,9.82,9.71,9.70,9.78
3,2021-01-07,9.82,9.71,9.70,9.78,9.81
4,2021-01-08,9.71,9.70,9.78,9.81,9.82
...,...,...,...,...,...,...
803,2024-02-20,10.16,10.28,10.36,10.26,10.23
804,2024-02-21,10.28,10.36,10.26,10.23,10.40
805,2024-02-22,10.36,10.26,10.23,10.40,10.20
806,2024-02-23,10.26,10.23,10.40,10.20,10.14


In [6]:

def modelo_train(new_df):

    # Importar herramientas:

    from sklearn.model_selection import train_test_split as tts 
    from sklearn.ensemble import RandomForestRegressor as RFR 
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP
    from sklearn.metrics import mean_squared_error as mse

    # Separar X e y:

    X = new_df.drop(['do','date'],axis=1)
    y = new_df.do

    # Ejecutar tts:

    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # Iniciar Modelos

    rfr=RFR()
    omp= OMP()

    # Entrenar

    rfr.fit(X_train, y_train)
    omp.fit(X_train, y_train)
     
    # Predecir

    y_pred_rfr=rfr.predict(X_test)
    y_pred_omp=omp.predict(X_test)

    # Mostrar rmse de cada modelo   
   
    return 'RMSE RFR ======',mse(y_test, y_pred_rfr, squared=False),'REMS OMP ======',mse(y_test, y_pred_omp, squared=False)

In [7]:
modelo_train(new_df)

('RMSE RFR ======',
 0.22692748393035736,
 'REMS OMP ======',
 0.20798462361777012)

In [8]:
def model(new_df):

    fecha_inicio = '2024-01-01'
    fecha_fin = '2024-02-26'

    # Filtrar el DataFrame
    mask = (new_df['date'] >= fecha_inicio) & (new_df['date'] <= fecha_fin)
    df_pred = new_df.loc[mask]

    df_pred_1 = df_pred[['d-4','d-3','d-2','d-1']]
    
    
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP

    omp= OMP()

    predicciones_y = []

    # Iterar sobre las 40 posiciones
    for i in range(40):
        # Seleccionar la porción correspondiente de X e y para esta iteración
        X_iteracion = new_df.drop(['date', 'do'], axis=1)[:-40+i]
        y_iteracion = new_df.do[:-40+i]

        # Ajustar el modelo
        omp.fit(X_iteracion, y_iteracion)

        # Seleccionar la fila correspondiente de df_pred para la predicción
        pred = df_pred_1[i:i+1]

        # Realizar la predicción y almacenarla en la lista
        y_pred_omp = omp.predict(pred)
        predicciones_y.append(y_pred_omp[0])   

    pred = pd.DataFrame(predicciones_y)

    pred.columns = ['prediccion']

    df_pred = df_pred.reset_index(drop=True)
    
    df_pred['pred'] = pred.prediccion.round(2)

    df_pred['error'] = df_pred.pred.round(2) - df_pred.do  

    return  df_pred

In [9]:
df_pred = model(new_df)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error
0,2024-01-02,10.59,10.45,10.44,10.32,10.37,10.32,-0.05
1,2024-01-03,10.45,10.44,10.32,10.37,10.27,10.37,0.10
2,2024-01-04,10.44,10.32,10.37,10.27,10.09,10.27,0.18
3,2024-01-05,10.32,10.37,10.27,10.09,10.15,10.09,-0.06
4,2024-01-08,10.37,10.27,10.09,10.15,10.28,10.15,-0.13
5,2024-01-09,10.27,10.09,10.15,10.28,10.40,10.28,-0.12
6,2024-01-10,10.09,10.15,10.28,10.40,10.10,10.40,0.30
7,2024-01-11,10.15,10.28,10.40,10.10,9.92,10.10,0.18
8,2024-01-12,10.28,10.40,10.10,9.92,9.90,9.92,0.02
9,2024-01-15,10.40,10.10,9.92,9.90,9.85,9.90,0.05


In [10]:
df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error
0,2024-01-02,10.59,10.45,10.44,10.32,10.37,10.32,-0.05
1,2024-01-03,10.45,10.44,10.32,10.37,10.27,10.37,0.10
2,2024-01-04,10.44,10.32,10.37,10.27,10.09,10.27,0.18
3,2024-01-05,10.32,10.37,10.27,10.09,10.15,10.09,-0.06
4,2024-01-08,10.37,10.27,10.09,10.15,10.28,10.15,-0.13
5,2024-01-09,10.27,10.09,10.15,10.28,10.40,10.28,-0.12
6,2024-01-10,10.09,10.15,10.28,10.40,10.10,10.40,0.30
7,2024-01-11,10.15,10.28,10.40,10.10,9.92,10.10,0.18
8,2024-01-12,10.28,10.40,10.10,9.92,9.90,9.92,0.02
9,2024-01-15,10.40,10.10,9.92,9.90,9.85,9.90,0.05


In [11]:
def escenario(x):

    import math

    df_pred['stock'] = (x/df_pred.do).apply(math.floor)

    df_pred['day_gap'] = df_pred.do.diff()

    df_pred['money'] = df_pred.stock*df_pred.day_gap

    df_pred['evolution'] = x + df_pred['money'].iloc[1:].cumsum().round(2)

    max_money_index = df_pred['money'].idxmax()
    
    df_pred['opti_real'] = x + df_pred['money'].iloc[max_money_index:].cumsum().round(2)

    min_money_index = df_pred['pred'].idxmin()
    
    df_pred['opti_pred'] = x + df_pred['money'].iloc[min_money_index:].cumsum().round(2)

    return df_pred

In [12]:
df_pred = escenario(100)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred
0,2024-01-02,10.59,10.45,10.44,10.32,10.37,10.32,-0.05,9,NaN,NaN,NaN,NaN,NaN
1,2024-01-03,10.45,10.44,10.32,10.37,10.27,10.37,0.10,9,-0.10,-0.90,99.10,NaN,NaN
2,2024-01-04,10.44,10.32,10.37,10.27,10.09,10.27,0.18,9,-0.18,-1.62,97.48,NaN,NaN
3,2024-01-05,10.32,10.37,10.27,10.09,10.15,10.09,-0.06,9,0.06,0.54,98.02,NaN,NaN
4,2024-01-08,10.37,10.27,10.09,10.15,10.28,10.15,-0.13,9,0.13,1.17,99.19,NaN,NaN
5,2024-01-09,10.27,10.09,10.15,10.28,10.40,10.28,-0.12,9,0.12,1.08,100.27,NaN,NaN
6,2024-01-10,10.09,10.15,10.28,10.40,10.10,10.40,0.30,9,-0.30,-2.70,97.57,NaN,NaN
7,2024-01-11,10.15,10.28,10.40,10.10,9.92,10.10,0.18,10,-0.18,-1.80,95.77,NaN,NaN
8,2024-01-12,10.28,10.40,10.10,9.92,9.90,9.92,0.02,10,-0.02,-0.20,95.57,NaN,NaN
9,2024-01-15,10.40,10.10,9.92,9.90,9.85,9.90,0.05,10,-0.05,-0.50,95.07,NaN,NaN


In [16]:
def dafatrame(df_pred,name):
    

    df_pred['name'] = name
    
    df_pred = df_pred[['name','date','do','pred','error','stock','day_gap','money','evolution','opti_real','opti_pred']]

    return df_pred

In [19]:
df= dafatrame(df_pred,'acerinox')

df

,name,date,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred
0,acerinox,2024-01-02,10.37,10.32,-0.05,9,NaN,NaN,NaN,NaN,NaN
1,acerinox,2024-01-03,10.27,10.37,0.10,9,-0.10,-0.90,99.10,NaN,NaN
2,acerinox,2024-01-04,10.09,10.27,0.18,9,-0.18,-1.62,97.48,NaN,NaN
3,acerinox,2024-01-05,10.15,10.09,-0.06,9,0.06,0.54,98.02,NaN,NaN
4,acerinox,2024-01-08,10.28,10.15,-0.13,9,0.13,1.17,99.19,NaN,NaN
5,acerinox,2024-01-09,10.40,10.28,-0.12,9,0.12,1.08,100.27,NaN,NaN
6,acerinox,2024-01-10,10.10,10.40,0.30,9,-0.30,-2.70,97.57,NaN,NaN
7,acerinox,2024-01-11,9.92,10.10,0.18,10,-0.18,-1.80,95.77,NaN,NaN
8,acerinox,2024-01-12,9.90,9.92,0.02,10,-0.02,-0.20,95.57,NaN,NaN
9,acerinox,2024-01-15,9.85,9.90,0.05,10,-0.05,-0.50,95.07,NaN,NaN


In [25]:
df.to_csv('acerinox.csv',index=False)